## Google Scholar Crawler
*DISCLAIMER: after retrieving 490 URLs, Google BLOCKED the IP address

*TODO: add selenium and add a manual step for opening a url and showing that we are NOT a robot

In [1]:
keyword_raw = "honeypot"

In [2]:
import cfscrape
from lxml import etree
import pandas as pd
import urllib
import re
from random import randint
from time import sleep

#Add selenium!!!! 
#change the user agent
#make a small break when google ask for captcha

header = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9',
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language' : 'nl-NL,nl;q=0.8,en-US;q=0.6,en;q=0.4',
    'Cache-Control' : 'max-age=0',
    'Connection': 'keep-alive',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.81 Safari/537.36'
    }

In [3]:
page = 1 
df = pd.DataFrame()
    
while True: 
    start_page = (page - 1) * 10
    #################################################################################
    keyword = urllib.parse.quote_plus("\""+keyword_raw+"\"")
    url="https://scholar.google.nl/scholar?start="+str(start_page)+"&q="+keyword+"&hl=nl&as_sdt=0,5"
    print(page, url)
    #################################################################################
    scraper = cfscrape.create_scraper()
    scraped_html=scraper.get(url,headers=header).content
    # print(scraped_html.decode("utf-8") )
    html = etree.HTML(scraped_html)
    #################################################################################    
    elements = html.xpath("//div[@class='gs_ri']")
    #################################################################################
    if len(elements) == 0:
        print("The code stopped for some reason!! Possibly Google blocked us!")
        print("\""+(''.join(html.xpath("//div[@id='infoDiv']/text()")))+"\"")
        break
    #################################################################################
    for element in elements:
        #################################################################################
        if len(element.xpath("h3[@class='gs_rt']/a/text()")) > 1:
            title = (keyword_raw+" ").join(element.xpath("h3[@class='gs_rt']/a/text()"))
        elif len(element.xpath("h3[@class='gs_rt']/a/text()")) == 1:
            title = element.xpath("h3[@class='gs_rt']/a/text()")[0]
        else:
            title = ""
        #################################################################################    
        link  = ''.join(element.xpath("h3[@class='gs_rt']/a/@href"))
        #################################################################################    
        abstract = ''.join(element.xpath("div[@class='gs_rs']/text()")).replace('\n','')
        #################################################################################
        reference = ''.join(element.xpath("div[@class='gs_a']/text()"))
        #################################################################################
        try:
            year = re.search(r"\d{4}", reference)[0]
        except:
            year = ""
        #################################################################################   
        df = df.append({'year':year,'title':title,'abstract':abstract,'reference':reference,'link':link},ignore_index=True)
    #################################################################################   
    print("sleeping...")
    sleep(randint(6,18))
    #################################################################################   
    page = page + 1       

1 https://scholar.google.nl/scholar?start=0&q=%22honeypot%22&hl=nl&as_sdt=0,5
sleeping...
2 https://scholar.google.nl/scholar?start=10&q=%22honeypot%22&hl=nl&as_sdt=0,5
sleeping...
3 https://scholar.google.nl/scholar?start=20&q=%22honeypot%22&hl=nl&as_sdt=0,5
sleeping...
4 https://scholar.google.nl/scholar?start=30&q=%22honeypot%22&hl=nl&as_sdt=0,5
sleeping...
5 https://scholar.google.nl/scholar?start=40&q=%22honeypot%22&hl=nl&as_sdt=0,5
sleeping...
6 https://scholar.google.nl/scholar?start=50&q=%22honeypot%22&hl=nl&as_sdt=0,5
sleeping...
7 https://scholar.google.nl/scholar?start=60&q=%22honeypot%22&hl=nl&as_sdt=0,5
sleeping...
8 https://scholar.google.nl/scholar?start=70&q=%22honeypot%22&hl=nl&as_sdt=0,5
sleeping...
9 https://scholar.google.nl/scholar?start=80&q=%22honeypot%22&hl=nl&as_sdt=0,5
sleeping...
10 https://scholar.google.nl/scholar?start=90&q=%22honeypot%22&hl=nl&as_sdt=0,5
sleeping...
11 https://scholar.google.nl/scholar?start=100&q=%22honeypot%22&hl=nl&as_sdt=0,5
sleeping.

## Exporting the output to a csv file

In [4]:
df.to_csv('honeypot_googlescholar_query_honeypot_20190722.csv')

## Further analysis

In [5]:
pd.options.display.max_colwidth = 100
from urllib.parse import urlparse

In [69]:
#Reading again the file in case it is a offline analysis
df = pd.read_csv('honeypot_googlescholar_query_honeypot_20190722.csv',index_col=0).dropna()
df.head()

,abstract,link,reference,title,year
0,A is a closely monitored network decoy serving several purposes: it can distract adversaries fro...,https://www.usenix.org/event/sec04/tech/full_papers/provos/provos_html,"- USENIX Security Symposium, 2004 - usenix.org",A Virtual honeypot Framework.,2004
1,"Spammers continually scan the Internet for open proxy relays: by using these open relays, they c...",https://ieeexplore.ieee.org/abstract/document/1264861/,"N Krawetz - IEEE Security & Privacy, 2004 - ieeexplore.ieee.org",Anti-honeypot technology,2004
2,Honeypots are closely monitored network decoys serving several purposes: they can distract adver...,https://www.researchgate.net/profile/Niels_Provos/publication/250395424_Honeyd_A_Virtual_Honeypo...,"- 10th DFN-CERT Workshop, Hamburg, Germany, 2003 - researchgate.net",Honeyd-a virtual honeypot daemon,2003
3,"The number of client-side attacks has grown significantly in the past few years, shifting focus ...",https://www.usenix.org/legacy/event/leet09/tech/full_papers/nazario/nazario.pdf,"- LEET, 2009 - usenix.org",PhoneyC: A Virtual Client honeypot .,2009
4,"Some attacks on honeypots are very frequent and repetitive. In addition, such repetitive attacks...",ftp://ftp.mirror.ac.za/www.honeynet.org/papers/individual/AusCERT_fullpaper_BIS.pdf,"F Pouget, - AusCERT Asia Pacific Information Technology …, 2004 - mirror.ac.za",-based forensics,2004


In [7]:
print("TOTAL ENTRIES:",len(df))

TOTAL ENTRIES: 393


In [70]:
source_distribution = df['link'].dropna().apply(lambda x: urlparse(x)[1].replace("www.","")).value_counts()
print("Distinct Source of Information:",len(source_distribution))
print()
print("Frequency of Source of Information:")
print(source_distribution.head(30))

Distinct Source of Information: 74

Frequency of Source of Information:
ieeexplore.ieee.org          141
link.springer.com             38
en.cnki.com.cn                22
pdfs.semanticscholar.org      21
patents.google.com            20
dl.acm.org                    17
citeseerx.ist.psu.edu         15
sciencedirect.com              9
researchgate.net               9
ro.ecu.edu.au                  5
usenix.org                     5
academia.edu                   5
cqvip.com                      5
onlinelibrary.wiley.com        5
tandfonline.com                4
arxiv.org                      4
search.ieice.org               3
pdiconnect.com                 3
infosecwriters.com             2
mecs-press.net                 2
apps.dtic.mil                  2
books.google.nl                2
spiedigitallibrary.org         2
eurecom.fr                     2
madoc.bib.uni-mannheim.de      1
docsdrive.com                  1
cscjournals.org                1
1is.us                         1
kris

The most important source of information are: 
- ieeexplore.ieee.org
- dl.acm.org
- link.springer.com
- pdfs.semanticscholar.org
- sciencedirect.com
- researchgate.net

Of course the RFCs of IETF are also very important:
- rfc-editor.org 
- tools.ietf.org
- datatracker.ietf.org

## Exporting information to csv files based on the source of information

## Crawling abstracts from IEEE

In [15]:
df[df['link'].str.contains("ieeexplore.ieee.org")].to_csv("honeypot_ieee.csv")

In [68]:
for link in df[df['link'].str.contains("ieeexplore.ieee.org")]['link']:
    print(link)
    #################################################################################
    scraper = cfscrape.create_scraper()
    scraped_html=scraper.get(link,headers=header).content
    # print(scraped_html.decode("utf-8") )
    html = etree.HTML(scraped_html)
    #################################################################################    
    elements = html.xpath("//script[@type='text/javascript']/text()")
    abstract = elements[2].split('"abstract":"')[2].split('","')[0]
    print(abstract)
    

https://ieeexplore.ieee.org/abstract/document/1264861/
Spammers continually scan the Internet for open proxy relays: by using these open relays, they can obscure their originating IP address and remain anonymous. However, when a spammer comes across a service on a honeypot, that honeypot can collect valuable information about the spammer's true identity and help unmask it. In response to the threat that honeypots pose to spammers, the first commercial anti-honeypot technology has surfaced: Send-Safe's Honeypot Hunter (www.send-safe. com) attempts to detect \"safe\" proxies for use with bulk-mailing tools. This honeypot-detection system's appearance, in association with other emerging spam tools, suggests three important trends: honeypots are affecting spammers, current honeypot technology is detectable, and more honeypot-identification systems are likely. The ability to detect a honeypot is unlikely to remain limited to spammers; other hostile or malicious groups could benefit from sim

Honeypots are computer systems that try to fool cyberattackers into thinking they are ordinary computer systems, when in fact they are designed solely to collect data about attack methods and thereby enable better defense against attackers. Honeypots are more effective the more ordinary they appear, but so far designers have just used intuition in designing them. So it is valuable to develop metrics for measuring the effectiveness of honeypot deception. We report on several software tools we have developed for assessing the effectiveness of honeypots, particularly a metric-calculating tool that summarizes a file system by a vector of 72 numbers. Comparison of vectors between fake and real systems can guide design of the fake. We show that this metric tool, applied to a detailed fake file system we constructed, confirms that it is convincing in most ways.
https://ieeexplore.ieee.org/abstract/document/4258527/
Voice over IP (VoIP) or telephony services over Internet announces a new revol

With the improvement of software security, attacks based on RPC vulnerabilities declined, however, attacks based on client application software vulnerabilities have increased. Such client application software includes Web browsers, email client and office. The spread of malware using these software vulnerabilities has become a severe threat to todaypsilas Internet. In allusion to this kind of threat, this paper designed an Internet malware collecting system based on client-side honeypot. This system can not only collect malware but also detect malicious Web site. It uses a unique network crawler based on client-side attack techniques to collect source of URL, and it collects URL and attachments from emails, then it creates software processes to open URL or files, and uses a device-drive monitor to detect malicious behaviors. It gives an alarm and locates the malicious file, and sends the malware coming through the Internet to the collecting server. We introduce the design and implement

This paper presents an intrusion detection module based on honey pot technology, which utilizes IP Trace back technique. By using the mobile agents, this module has the capability of distributed detection and response, the whole detection module can be extended with convenience and be configured dynamically and flexibly. By using honey pot technology, this module traces the intrusion source farthest.
https://ieeexplore.ieee.org/abstract/document/6866707/
The wide spread of cyber-attacks made the need of gathering as much information as possible about them, a real demand in nowadays global context. The honeypot systems have become a powerful tool on the way to accomplish that. Researchers have already focused on the development of various honeypot systems but the fact that their administration is time consuming made clear the need of self-adaptive honeypot system capable of learning from their interaction with attackers. This paper presents a self-adaptive honeypot system we are develop

In this paper, we describe the development of J-Honeypot, a Java-based network deception tool with a Web-based monitoring interface and a rule-based intrusion detection engine. Our J-Honeypot is based on the Honeyd program by Niels Provos, but features important enhancements. First, since J-Honeypot is written in Java, it can potentially run on various platforms, Windows or UNIX, workstations or handheld devices. Second, J-Honeypot has a rule-based intrusion detection engine, whose design is guided by the analysis of real world attack data. The attack data was collected by opening up our J-Honeypot to solicit possible attacks. Third, J-Honeypot includes a Web-based monitoring tool that is easily accessible and can help network administrators better understand network traffic and possible attacks.
https://ieeexplore.ieee.org/abstract/document/5587041/
Honeypot is an effective security tool, which is intended to be attacked and compromised to gain more information about the attacker and 

...

With the growing sophistication of computer worms, information security has become a prime concern for individuals, community and organizations. Traditional signature based IDS, though effective for known attacks but failed to handle the unknown attack promptly. This paper describes a novel honeypot system which capture worm based on their characteristics of self replication. We introduce combination of unlimited and limited outbound connections to capture different payload of single or multiple worms. The proposed system isolate the suspicious traffic and able to effectively control the malicious traffic and capture most useful information regarding the worm's activities, without attacker's knowledge. Our system will be used for critical study of structure and behavior of most sophisticated worms and then forwards the necessary input to Signature Generation Module for automatically generating signature of unknown worms. Our attempt is to generate signature of unknown especially polymo

The data processing capabilities of MapReduce systems pioneered with the on-demand scalability of cloud computing have enabled the Big Data revolution. However, the data controllers/owners worried about the privacy and accountability impact of storing their data in the cloud infrastructures as the existing cloud computing solutions provide very limited control on the underlying systems. The intuitive approach - encrypting data before uploading to the cloud - is not applicable to MapReduce computation as the data analytics tasks are ad-hoc defined in the MapReduce environment using general programming languages (e.g, Java) and homomorphic encryption methods that can scale to big data do not exist. In this paper, we address the challenges of determining and detecting unauthorized access to data stored in MapReduce based cloud environments. To this end, we introduce alarm raising honeypots distributed over the data that are not accessed by the authorized MapReduce jobs, but only by the at

With the rapid spread of the Internet, information technologies (ITs) and inadequate basic security of computer systems, it is only a matter of time when the computer systems will be attacked after revealing its existing vulnerabilities and programme weak points. At present the presence of such a potential threat agitates a greater and greater interest in various possibilities for enhancing its security. Currently exerted defence system based on the use of firewall, antivirus software and Intrusion Detection System (IDS) represents a necessary minimum. This traditional approach to security of computer systems is primarily oriented to the defence against already existing threats. It is essential to adapt the security system to this trend as the current threats have much more aggressive forms. The use of decoy based security technology - Honeypot - represents an adequate form of defence. However, even this technology is not a “security panacea” as its 100% functionality is proportionally

Security issues and challenges constitute, undoubtedly, one of the major risks of using cloud computing services. There are approaches that address security requirements, nevertheless there are no guarantees that security breaches are not going to occur in this complex infrastructure. Coordinated and sophisticated attacks pose a significant threat to cloud environments and vendors need to be prepared to prevent, detect, and recover from them. In this paper, the focus of the discussion is on how honeypots, a nontraditional security technology not adequately represented in the protection mechanisms pool, could be deployed in the cloud to allow the analysis of attack patterns. To be more specific, CloudHoneyCY is presented, an open-source framework that supports a collection of low-interaction and high-interaction honeypots deployed in the cloud infrastructure with the purpose of collecting and analyzing attack data that aids in constructing attack profiles.
https://ieeexplore.ieee.org/ab

Wireless sensor networks (WSN) are entering the daily life of many. Like wireless LAN before, these are a new playground for developers, but also offer new ways for misuse of this technology. For the improvement of security and to get more valuable data about new attacks, a honeypot for WSNs is in development. This paper presents an effectiveness analysis of a honeypot for WSN and shows detection capabilities in the categories of known and unknown attacks.
https://ieeexplore.ieee.org/abstract/document/8249458/
This paper proposes an implementation of honeypot that detects and reports telnet attacks on Internet-of-Things (IoT) devices. The honeypot operates with manual and Mirai-based attacks. A multi-component design is implemented in order to attain sufficient exposure to malicious traffic and security of collected data. Settings and additional files needed to run the honeypot are explained. Honeypot is tested using Mirai and results are discussed. After that, conclusion and direction

In [41]:
df[df['link'].str.contains("link.springer.com")].to_csv("flowspec_springer.csv")

In [42]:
df[df['link'].str.contains("pdfs.semanticscholar.org")].to_csv("flowspec_semanticscholar.csv")

In [45]:
df[df['link'].str.contains("dl.acm.org")].to_csv("flowspec_acm.csv")

## Lessons learned:
- remove papers that have in the title or description/abstract the list of words: [rsvp, qos, quality of service, mpls, intserv, diffserv, multicast]